In [138]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Load DataFrame

In [139]:
df = pd.read_csv('../../raw_data/df.csv')
"""df = df.drop(columns='Unnamed: 0')
df['Day'] = pd.to_datetime(df['Day'])
df.set_index('Day', inplace=True)
df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})
df['day'] = df.index # we will use this in our column transformer
df = df.drop(columns = ['fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks'])"""

'df = df.drop(columns=\'Unnamed: 0\')\ndf[\'Day\'] = pd.to_datetime(df[\'Day\'])\ndf.set_index(\'Day\', inplace=True)\ndf = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})\ndf[\'day\'] = df.index # we will use this in our column transformer\ndf = df.drop(columns = [\'fb_impressions\', \'fb_clicks\', \'google_impressions\', \'google_clicks\', \'tt_impressions\', \'tt_clicks\'])'

In [140]:
df

,Unnamed: 0,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks,tt_costs,tt_impressions,tt_clicks
0,0,2021-07-01,10,414.85,257.01,36200.0,129.0,1.17,39.0,20.0,0.00,0.0,0.0
1,1,2021-07-02,6,229.95,250.30,33040.0,150.0,0.84,14.0,7.0,0.00,0.0,0.0
2,2,2021-07-03,10,459.89,248.59,33031.0,129.0,0.72,12.0,6.0,0.00,0.0,0.0
3,3,2021-07-04,17,508.36,258.05,37463.0,119.0,0.07,13.0,5.0,0.00,0.0,0.0
4,4,2021-07-05,13,399.88,257.47,35963.0,142.0,0.38,34.0,16.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,780,2023-08-20,294,13616.97,2148.53,555740.0,1366.0,466.56,70.0,801.0,368.86,199939.0,644.0
781,781,2023-08-21,356,15348.42,2210.01,595105.0,1284.0,400.58,51.0,782.0,391.14,186956.0,494.0
782,782,2023-08-22,150,6054.53,1335.84,340325.0,839.0,289.46,50.0,619.0,218.61,134032.0,398.0
783,783,2023-08-23,130,4810.67,1315.81,337607.0,841.0,275.86,52.0,537.0,90.21,43483.0,133.0


# Baseline Model

In [141]:
baseline = train_df.shift(1) # baseline['total_sales'] is equivalent to y_pred

y_pred = baseline['total_sales']
y_true = train_df['total_sales']

# MSE
mse = ((y_true - y_pred) ** 2).mean()

# Calculate TSS (Total Sum of Squares)
mean_y_true = np.mean(y_true)
tss = np.sum((y_true - mean_y_true)**2)

# Calculate RSS (Residual Sum of Squares)
rss = np.sum((y_true - y_pred)**2)

# Calculate R-squared (Coefficient of Determination)
r_squared = 1 - (rss / tss)

f"{mse} is our MSE to beat; {r_squared} is our R squared to beat!"

'595453.442743578 is our MSE to beat; 0.6398209085770498 is our R squared to beat!'

In [142]:
class DatasetTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df = df.drop(columns='Unnamed: 0')
        df['Day'] = pd.to_datetime(df['Day'])
        df.set_index('Day', inplace=True)
        df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})
        df = df.drop(columns = ['orders','fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks'])
        n_splits = 5  # Number of splits for cross-validation
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for train_index, test_index in tscv.split(df):
            train_df = df.iloc[train_index]
            test_df = df.iloc[test_index]

        y_train = train_df['total_sales']
        X_train = train_df.drop(columns = ['total_sales'])


        y_test = test_df['total_sales']
        X_test = test_df.drop(columns = ['total_sales'])
        return train_df, y_train, test_df, y_test

In [157]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error, r2_score

# Define a custom transformer to calculate MSE and R-squared
class MetricsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return X  # No transformation needed
    
    def score(self, X, y):
        baseline = train_df.shift(1)
        y_pred = baseline['total_sales']
        y_true = train_df['total_sales']
        y_pred[0] = y_true[0]
        mse = mean_squared_error(y_true, y_pred)
        r_squared = r2_score(y_true, y_pred)
        return {'mse': mse, 'r_squared': r_squared}

In [159]:
import pandas as pd
import numpy as np

class BaselineModel:
    def fit(self, X, y=None):
        # The baseline model doesn't require any training, so this method is empty
        return self
    
    def predict(self, X):
        # Calculate predictions based on the specified period (default is 1)
        df = X.copy()
        predictions = X['total_sales'].shift(1)
        predictions.bfill(inplace=True)
        return predictions

In [160]:
# Create a pipeline with your custom transformer
pipeline = Pipeline([
    ('df_prep', DatasetTransformer()),
    ('metrics', MetricsTransformer()),
    ('model', BaselineModel())
])

In [162]:
# Fit the pipeline on the training data
pipeline.fit(train_df, y_train)

# Print the results
f"R-squared: {metrics['r_squared']}"

'R-squared: 0.6398209085770498'

In [165]:
pipeline.predict(train_df)

AttributeError: 'tuple' object has no attribute 'copy'